# Connecting to CUDA Grünau

Unfortunately, I tried out Google Colab, but it did not work out.
Even after fixing all not working things (that work everywhere else except colab), its capacity is too limited and they actually do not write after how many hours they interrupt process ("because everytime it is different").

So, if you are not going to use CUDA Grünau, skip this part.

If you are interested, which code is working in Colab, it is in Appendix. It runs only on small datasets (that processes are not interrupted).

In [2]:
# For gruenau: For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [24]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [37]:
torch.cuda.get_device_name(2)

'Quadro RTX 6000'

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Nov__3_17:16:49_PDT_2023
Cuda compilation tools, release 12.3, V12.3.103
Build cuda_12.3.r12.3/compiler.33492891_0


In [19]:
!nvidia-smi

Fri Apr 19 16:50:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:3B:00.0 Off |                    0 |
| N/A   28C    P0             35W /  250W |     819MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
gpu_index = "1"

# 1. Prepare datasets

In [3]:
import pandas as pd
import numpy as np
import os

In [2]:
# Create a folder named "datasets" if it doesn't exist
folder_name = "datasets"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

df = pd.read_csv("top_5_countries.csv", index_col=0, parse_dates=True)
# Reset index for Data Loader
df.reset_index(inplace=True)
df = df.iloc[:,:-6]

In [3]:
top_5_countries = ['DE', 'GB', 'ES', 'FR', 'IT']

# Split and save the datasets
for country_prefix in top_5_countries:
    # Filter columns with the specified prefix
    country_columns = [col for col in df.columns if col.startswith(country_prefix)]
    
    # Insert the date column at the beginning of every dataset
    country_columns.insert(0,"date")
    country_df = df[country_columns]
    
    # Save the DataFrame to a CSV file
    file_name = f"datasets/{country_prefix}_data.csv"
    country_df.to_csv(file_name, index=False)


# 2. Informer

In [13]:
# https://colab.research.google.com/drive/1rv2rKwQqgoHDNjXtRoAEWZ2ATz0gGAKu?usp=sharing#scrollTo=yu6zzic9t_Cz
# Popen: https://colab.research.google.com/github/aviadr1/learn-python/blob/master/content/13_multiprocessing/notebooks/os_system_subprocess.ipynb
import subprocess
import os
# parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_to_run_file = "/vol/cs-hu/riabchuv/hu-home/my_work/TSLibrary/run.py"

def run_output(path_to_run_file, model_arguments):
    try:
        # Execute the script and capture the output
        command = ["python", "-u", path_to_run_file] + model_arguments
        output = subprocess.check_output(command, universal_newlines=True)
    except subprocess.CalledProcessError as e:
        output = e.output  

    return output

In [ ]:
"""
import subprocess
import os
# parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_to_run_file = "/content/my_work/TSLibrary/run.py"

def run_output(path_to_run_file, model_arguments):
    try:
        # Define command and options wanted
        command = "python"
        options = "-u"
        # Run the shell command directly in Colab
        output = !{command} {options} {path_to_run_file} {model_arguments}

    except subprocess.CalledProcessError as e:
        output = e.output

    return output

def run_output(path_to_run_file, model_arguments):
    try:
        # Execute the script using the %run magic command
        output = %run -i {path_to_run_file} {model_arguments}

    except Exception as e:
        output = str(e)

    return output
"""

In [4]:
# Seems to work in colab
import subprocess

def run_output(path_to_run_file, model_arguments):
    try:
        # Construct the command to execute the script with required and model arguments
        command = ["python", "-u", path_to_run_file] + model_arguments
        # Execute the script and capture the output
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        # Check if there's any error in the process
        if process.returncode != 0:
            output = stderr.decode("utf-8")
        else:
            output = stdout.decode("utf-8")
    except Exception as e:
        output = str(e)
    return output

# Colab

In [ ]:
!git clone https://github.com/well0203/my_work.git
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -r /content/my_work/TSLibrary/requirements.txt
#!pip install sktime
#!pip install reformer-pytorch==1.4.4
# Drive python version 3.10.6, therefore torch==1.7.1 does not work

In [40]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'/vol/cs-hu/riabchuv/hu-home/my_work/results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__24_DE_Informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30121
val 4321
test 8617
	iters: 100, epoch: 1 | loss: 0.5979588
	speed: 0.1078s/iter; left time: 1003.5939s
	iters: 200, epoch: 1 | loss: 0.3988746
	speed: 0.0981s/iter; left time: 903.5919s
	iters: 300, epoch: 1 | loss: 0.4951838
	speed: 0.0980s/iter; left time: 892.9673s
	iters: 400, epoch: 1 | loss: 0.3251971
	speed: 0.0979s/iter; left time: 882.0710s
	iters: 500, epoch: 1 | loss: 0.2972392
	speed: 0.0979s/iter; left time: 872.1553s
	iters: 600, epoch: 1 | loss: 0.3141234
	speed: 0.0983s/iter; left time: 865.7961s
	iters: 700, epoch: 1 | loss: 0.3582392
	speed: 0.0980s/iter; left time: 853.8956s
	iters: 800, epoch: 1 | loss: 0.3090230
	speed: 0.0979s/iter; left time: 842.9842s
	iters: 900, epoch: 1 | loss: 0.2600375
	speed: 0.0980s/iter; left time: 833.7338s
Epoch: 1 running time: 1.5546314120292664 mi

# Test

In [5]:
dp='GB_data_small.csv'

In [ ]:
/vol/cs-hu/riabchuv/hu-home/my_work/datasets/GB_data_small.csv

In [10]:
current_path = os.getcwd() + "/datasets/"
current_path

'/vol/cs-hu/riabchuv/hu-home/my_work/datasets/'

In [11]:
pd.read_csv(os.path.join(current_path, dp), index_col=0, parse_dates=True)

,GB_UKM_load_actual_entsoe_transparency,GB_UKM_solar_generation_actual,GB_UKM_wind_generation_actual,GB_UKM_wind_offshore_generation_actual,GB_UKM_wind_onshore_generation_actual
date,,,,,
2015-10-26 00:00:00,30680.0,0.0,5348.0,1885.0,3463.0
2015-10-26 01:00:00,29218.0,0.0,5194.0,1810.0,3383.0
2015-10-26 02:00:00,28016.0,0.0,4389.0,1756.0,2633.0
2015-10-26 03:00:00,27402.0,0.0,5104.0,1687.0,3417.0
2015-10-26 04:00:00,27490.0,0.0,5206.0,1749.0,3456.0
...,...,...,...,...,...
2015-11-04 19:00:00,49595.0,0.0,1012.0,584.0,429.0
2015-11-04 20:00:00,46550.0,0.0,1246.0,792.0,455.0
2015-11-04 21:00:00,42752.0,0.0,1441.0,915.0,526.0


In [39]:
# 5.413828869660695 min. without test
# 6.30 min with test
import time
start = time.time()

current_path = os.getcwd() + "/datasets/"

datasets = ['GB_data_small.csv', 'GB_data_small.csv']
num_cols = ["5", "5"]
pred_len = "10"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
            "--task_name", "long_term_forecast",
            "--is_training", "1", #True
            "--root_path", current_path, #"/content/my_work/datasets/",
            "--data_path", dataset,
            #"--train_epochs", "1",
            "--model_id", model_id,
            "--model", model,
            "--data", "custom", # This ensures a 70%,10%,20% train,val,test split see data_provider/data_loader.py
            "--features", "M", # Multivariate
            "--seq_len", "10",
            "--label_len", "5",
            "--pred_len", pred_len,
            "--e_layers", "3", # Hyperparameters as in original model
            "--d_layers", "2",
            "--factor", "3",
            #"--gpu", gpu_index,
            # '--use_multi_gpu', 
            "--enc_in", num_cols[i],
            "--dec_in", num_cols[i],
            "--c_out", num_cols[i],
            "--des", "Exp",
            "--itr", "2",
        ]
    int_start = time.time()
    model_output = run_output(path_to_run_file, model_arguments)

    # folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'/vol/cs-hu/riabchuv/hu-home/my_work/results/{model}/'

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


end = time.time()

print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__10_GB_Informer_custom_ftM_sl10_ll5_pl10_dm512_nh8_el3_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 149
val 15
test 39
Epoch: 1 running time: 0.039925122261047365 min.
Epoch: 1, Steps: 4 | Train Loss: 1.0024538 Vali Loss: nan Test Loss: 1.2172809
Validation loss decreased (inf --> nan).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 running time: 0.01469890276590983 min.
Epoch: 2, Steps: 4 | Train Loss: 0.6911948 Vali Loss: nan Test Loss: 1.4431949
Validation loss decreased (nan --> nan).  Saving model ...
Updating learning rate to 5e-05
Epoch: 3 running time: 0.014795935153961182 min.
Epoch: 3, Steps: 4 | Train Loss: 0.5788186 Vali Loss: nan Test Loss: 1.0701528
Validation loss decreased (nan --> nan).  Saving model ...
Updating learning rate to 2.5e-05
Epoch: 4 running time: 0.015081985791524252 min.
Epoch: 4, Steps: 4 | Train Loss: 0.5235551 Vali Loss: nan Test Loss: 0.9722701
Validation l

In [19]:
# metrics
np.load("/Users/valentyna/Documents/Master_thesis_new/results/long_term_forecast_1_Informer_custom_ftM_sl10_ll5_pl10_dm512_nh8_el3_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0/metrics.npy")

array([  0.86847895,   1.1777946 ,   1.0852624 ,   3.2318454 ,
       769.5208    ], dtype=float32)

In [21]:
# preds
np.load("/Users/valentyna/Documents/Master_thesis_new/results/long_term_forecast_1_Informer_custom_ftM_sl10_ll5_pl10_dm512_nh8_el3_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0/pred.npy").shape

(39, 10, 5)